In [ ]:
pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00


In [1]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

checkpoint = "HuggingFaceM4/idefics-9b-instruct"
model = IdeficsForVisionText2Text.from_pretrained(checkpoint, torch_dtype=torch.bfloat16).to(device)
processor = AutoProcessor.from_pretrained(checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/99.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/7.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=False)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)

In [3]:
# We feed to the model an arbitrary sequence of text strings and images. Images can be either URLs or PIL Images.
prompts = [
    [
        "User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger, Disgust.",
        "https://uploads5.wikiart.org/images/adriaen-brouwer/farmers-fight-party.jpg",
        "<end_of_utterance>",

        "\nAssistant: Disgust.<end_of_utterance>",

        "\nUser:",
        "https://uploads3.wikiart.org/images/vincent-van-gogh/still-life-with-basket-and-six-oranges-1888.jpg",
        "These fruits look bright, sweet, and juicy. The dark orange colors of the fruit compliment the light orange of the basket nicely. The oranges on the basket make for a quaint symbol of a harvest. This decorative basket of oranges looks like an appetizing table centerpiece that makes me want to eat one of the oranges right now. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger, Disust.<end_of_utterance>",

        "\nAssistant:",
    ],
]

# --batched mode
inputs = processor(prompts, add_end_of_utterance_token=False, return_tensors="pt").to(device)
# --single sample mode
# inputs = processor(prompts[0], return_tensors="pt").to(device)

# Generation args
exit_condition = processor.tokenizer("<end_of_utterance>", add_special_tokens=False).input_ids
bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=200)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
for i, t in enumerate(generated_text):
    print(f"{i}:\n{t}\n")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


0:
User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger, Disgust. 
Assistant: Disgust. 
User: These fruits look bright, sweet, and juicy. The dark orange colors of the fruit compliment the light orange of the basket nicely. The oranges on the basket make for a quaint symbol of a harvest. This decorative basket of oranges looks like an appetizing table centerpiece that makes me want to eat one of the oranges right now. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger, Disust. 
Assistant: Contentment.



In [ ]:
# We feed to the model an arbitrary sequence of text strings and images. Images can be either URLs or PIL Images.
prompts = [
    [
        "User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "https://uploads5.wikiart.org/images/adriaen-brouwer/farmers-fight-party.jpg",
        "<end_of_utterance>",

        "\nAssistant: Disgust.<end_of_utterance>",

        "\nUser:",
        "https://uploads3.wikiart.org/images/adriaen-brouwer/portrait-of-a-man.jpg",
        "The man seems very content in this portrait. The man's face looks happy. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",

        "\nAssistant:",
    ],
]

# --batched mode
inputs = processor(prompts, add_end_of_utterance_token=False, return_tensors="pt").to(device)
# --single sample mode
# inputs = processor(prompts[0], return_tensors="pt").to(device)

# Generation args
exit_condition = processor.tokenizer("<end_of_utterance>", add_special_tokens=False).input_ids
bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=300)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
for i, t in enumerate(generated_text):
    print(f"{i}:\n{t}\n")


0:
User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: The man seems very content in this portrait. The man's face looks happy. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.
Assistant: Contentment.



In [ ]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

# Load the CSV file
df = pd.read_csv('final_data.csv')

selected_columns = df[['utterance', 'Link', 'emotion']]

for index, row in selected_columns.iterrows():
    utterance = row['utterance']
    link = row['Link']
    gold_label = row['emotion']

    prompts = [
        [
            "User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
            "https://uploads5.wikiart.org/images/adriaen-brouwer/farmers-fight-party.jpg",
            "<end_of_utterance>",

            "\nAssistant: Disgust.<end_of_utterance>",

            "\nUser:",
            link, 
            f"{utterance} Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",

            "\nAssistant:",
        ],
    ]

    # --batched mode
    inputs = processor(prompts, add_end_of_utterance_token=False, return_tensors="pt").to(device)
    # --single sample mode
    # inputs = processor(prompts[0], return_tensors="pt").to(device)

    # Generation args
    exit_condition = processor.tokenizer("<end_of_utterance>", add_special_tokens=False).input_ids
    bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

    generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=300)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
    for i, t in enumerate(generated_text):
        print(f"The {i} example:\n{t}\n")



The 0 example:
User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: Lost in a city and finding myself in a bad area of abandoned buildings. Because the black & white overtones and the objects that somewhat resemble headstones in a cemetery. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.
Assistant: Fear.

The 0 example:
User: the figures are grotesque and animal-like and the scene looks chaotic The figures appear dirty. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger. 
Assistant: Disgust. 
User: looks like some sort of creature from a scary film ghostly figure Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.
Assistant: Fear.

The 0 example:
User: the figures are grotesque and animal-like and the scene looks 

KeyboardInterrupt: 

In [ ]:
selected_columns

,utterance,Link,emotion
0,Lost in a city and finding myself in a bad are...,https://uploads7.wikiart.org/images/aaron-sisk...,fear
1,looks like some sort of creature from a scary ...,https://uploads5.wikiart.org/images/aaron-sisk...,fear
2,The black sharp lines running throughout the c...,https://uploads7.wikiart.org/images/aaron-sisk...,fear
3,Foot looks weird and deformed. I hate feet,https://uploads6.wikiart.org/images/aaron-sisk...,disgust
4,"it looks like an eye staring at me, or like a ...",https://uploads3.wikiart.org/images/aaron-sisk...,fear
...,...,...,...
95,they're killing a man with a knife and the dar...,https://uploads7.wikiart.org/images/adriaen-va...,fear
96,This person just looks funny and it seems like...,https://uploads1.wikiart.org/images/adriaen-va...,amusement
97,feels gather at night. happy. I like the warm ...,https://uploads8.wikiart.org/images/adriaen-va...,contentment
98,Something bad is hapening Are they hiding? Sh...,https://uploads6.wikiart.org/images/adriaen-va...,fear
